In [1]:
import os 
import datetime
import pandas as pd

from typing import List
from sqlalchemy import create_engine, text

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
hostname = os.environ["HOSTNAME"]
database = os.environ["DATABASE"]
port = os.environ["PORT"]

db_credentials = f"postgresql://{username}:{password}@{hostname}:{port}/{database}"
engine = create_engine(db_credentials)

In [4]:
sql_query = text(
f"""
SELECT
    studies.nct_id,
    MAX(studies.brief_title) AS brief_title,
    MAX(studies.official_title) AS official_title,
    STRING_AGG(DISTINCT baseline_measurements.description, ' ') AS baseline_measurements,
    STRING_AGG(DISTINCT brief_summaries.description, ' ') AS brief_summaries,
    STRING_AGG(DISTINCT detailed_descriptions.description, ' ') AS detailed_descriptions,
    MAX(eligibilities.criteria) AS criteria, 
    MAX(eligibilities.gender) AS gender, 
    MAX(eligibilities.minimum_age) AS minimum_age, 
    MAX(eligibilities.maximum_age) AS maximum_age, 
    MAX(facilities.name) AS facilities, 
    MAX(facilities.city) AS city, 
    MAX(facilities.state) AS state, 
    MAX(facilities.zip) AS zip, 
    MAX(facilities.country) AS country, 
    MAX(participant_flows.recruitment_details) AS recruitment_details, 
    MAX(participant_flows.pre_assignment_details) AS pre_assignment_details, 
    MAX(studies.study_type) AS study_type
FROM 
    ctgov.studies
INNER JOIN ctgov.baseline_measurements ON baseline_measurements.nct_id = studies.nct_id 
INNER JOIN ctgov.brief_summaries ON brief_summaries.nct_id = studies.nct_id 
INNER JOIN ctgov.detailed_descriptions ON detailed_descriptions.nct_id = studies.nct_id 
INNER JOIN ctgov.eligibilities ON eligibilities.nct_id = studies.nct_id 
INNER JOIN ctgov.facilities ON facilities.nct_id = studies.nct_id 
INNER JOIN ctgov.participant_flows ON participant_flows.nct_id = studies.nct_id 
GROUP BY studies.nct_id;
"""
)

In [5]:
# Execute the SQL query and create a pandas DataFrame from the result
df = pd.read_sql_query(
    sql_query,
    con=engine,
    params={} # type: ignore
)
df

,nct_id,brief_title,official_title,baseline_measurements,brief_summaries,detailed_descriptions,criteria,gender,minimum_age,maximum_age,facilities,city,state,zip,country,recruitment_details,pre_assignment_details,study_type
0,NCT00000143,Studies of Ocular Complications of AIDS (SOCA)...,Studies of Ocular Complications of AIDS (SOCA)...,None,"To compare the newest CMV retinitis drug, cido...",Cytomegalovirus (CMV) is among the most freque...,Inclusion criteria:\n\nAge 13 years or older\n...,All,13 Years,N/A,"University of South Florida, MDC Box 21",Tampa,Texas,94143,United States,June 1997,None,Interventional
1,NCT00000378,Antidepressant Treatment of Melancholia in Lat...,Antidepressant Treatment of Melancholia in Lat...,None,The purpose of this study is to compare the sa...,To compare the efficacy and safety of a select...,Inclusion Criteria:\n\n-\n\nPatients must have...,All,60 Years,95 Years,1051 Riverside Drive,New York,New York,10032,United States,None,None,Interventional
2,NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Action to Control Cardiovascular Risk in Diabe...,None,The purpose of this study is to prevent major ...,"BACKGROUND:\n\nCurrently, about 17 million Ame...",Inclusion Criteria:\n\nDiagnosed with type 2 d...,All,40 Years,79 Years,Wake Forest University,Winston-Salem,Washington,98195,United States,All participants had established type 2 diabet...,Eligible participants provided evidence of abi...,Interventional
3,NCT00001151,"Studies With 1,25-Dihydroxycholecalciferol","Studies With 1,25-Dihydroxycholecalciferol",None,Vitamin D in the diet undergoes changes in the...,"Patients with extreme resistance to 1,25-dihyd...",INCLUSION CRITERIA:\n\nPatients with hereditar...,All,N/A,N/A,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,None,None,Interventional
4,NCT00001213,Cysteamine Eye Drops to Treat Corneal Crystals...,Trial of Topical Cysteamine in the Treatment o...,Although 328 participants were initially enrol...,Cystinosis is an inherited disease that result...,"Protocol 86-EI-0062 began as a randomized, dou...",INCLUSION CRITERIA:\n\nPatients must have a do...,All,2 Years,N/A,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,None,None,Interventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34978,NCT05490771,Testing Copanlisib as a Potential Targeted Tre...,Phase II Study of Copanlisib in Patients With ...,None,This phase II MATCH treatment trial identifies...,PRIMARY OBJECTIVE:\n\nI. To evaluate the propo...,Inclusion Criteria:\n\nPatients must have met ...,All,18 Years,N/A,ECOG-ACRIN Cancer Research Group,Philadelphia,Pennsylvania,19103,United States,"Subprotocol Z1F was activated on June 20, 2018...",The PIK3CA mutations status was determined by ...,Interventional
34979,NCT05502081,Clinical Study to Compare Efficacy and Safety ...,Clinical Study to Evaluate the Possible Effica...,0. Uninfected\n\nAmbulatory mild disease\n\nAs...,Introduction:\n\nCorona Virus induced disease ...,I. INTRODUCTION\n\n1.1. COVID-19 overview and ...,Inclusion Criteria:\n\nage more than 12 years ...,All,12 Years,N/A,El-gomhoria St,Mansoura,El-dkhalia,050,Egypt,from 1/11/2021 to 29/5/2022 at isolation hospi...,assignment is applied after admission of parti...,Interventional
34980,NCT05594173,Chewing and Oral Processing of Solid Food,Chewing and Oral Processing of Solid Food in H...,None,Food texture modification is commonly used as ...,Aim: To explore chewing and oral processing be...,Inclusion Criteria:\n\nHealthy adults under ag...,All,18 Years,60 Years,Toronto Rehabilitation Institute - University ...,Toronto,Ontario,M5G 2A2,Canada,None,None,Observational
34981,NCT05607147,Rutgers Pilot for Dental Health Care Worker SA...,Rutgers Pilot for Pragmatic Return to Effectiv...,None,10 asymptomatic DHCWs in the Oral Medicine cli...,10 asymptomatic DHCWs in the Oral Medicine cli...,Inclusion Criteria:\n\nDent

In [6]:
len(df)

34983

In [7]:
df.set_index(df['nct_id'], inplace=True)
df.drop(columns=['nct_id'], inplace=True)


In [8]:
output = "ctgov"
timestamp = datetime.datetime.now().strftime("%Y%m%d")
file_name = f"{output}_{timestamp}.csv"
# Write the data to output filename
df.to_csv(file_name)